In [486]:
import pandas as pd
import time
import numpy as np
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [603]:
uh_df = pd.read_csv('../input/unit-history.csv.gz')
full_uh_df = uh_df.copy()

In [488]:
start_col = 'Start.Date'
end_col = 'End.Date'
unit_col = 'Unit'
uid_col = 'UID'
lower_bound = '2000-01-01'
upper_bound = ''
cap_date = ''

In [489]:
uh_df.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
1,NaN,2016-06-05,1,3.0
2,NaN,2007-03-29,2,15.0
3,1971-01-15,1970-06-15,3,22.0
4,1976-08-18,1971-01-16,3,4.0


In [490]:
uh_df.shape

(107786, 4)

In [491]:
uh_df[end_col] = pd.to_datetime(uh_df[end_col])
uh_df[start_col] = pd.to_datetime(uh_df[start_col])

In [492]:
if upper_bound:
    upper_bound = pd.to_datetime(upper_bound)
    assert isinstance(upper_bound, pd.Timestamp), 'upper bound given is not a proper date'
else:
    upper_bound = pd.to_datetime(pd.date_range(start = max(uh_df[start_col]),
                                            periods=2, freq='D'))[1]
uh_df = uh_df[uh_df[start_col] <= upper_bound]
uh_df.loc[(uh_df[end_col].isnull()) |
          (uh_df[end_col] > upper_bound),
          end_col] = upper_bound


In [493]:
uh_df.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
1,2016-10-10,2016-06-05,1,3.0
2,2016-10-10,2007-03-29,2,15.0
3,1971-01-15,1970-06-15,3,22.0
4,1976-08-18,1971-01-16,3,4.0


In [494]:
if lower_bound:
    uh_df = uh_df[uh_df[end_col] >= lower_bound]

In [495]:
uh_df.shape

(55265, 4)

In [496]:
def expand_times(sd,ed,unit,uid):
    outdf = pd.DataFrame(pd.date_range(start=sd, end=ed, freq='D', name='Date'))
    outdf.insert(1, 'Unit', unit)
    outdf.insert(2, 'UID', uid)
    event = [0] * outdf.shape[0]
    event[0] = 1
    event[-1] = -1
    outdf.insert(3, 'Event', event)
    return outdf

In [497]:
r = uh_df.loc[0]
sd = r[start_col]
ed = r[end_col]
unit = r[unit_col]
uid = r[uid_col]

In [498]:
outdf = {'Dates' : pd.date_range(start=sd, end=pd.to_datetime('2007-04-04'), freq='D', name='Date')}
rows = len(outdf['Dates'])
outdf['Unit'] = unit
outdf['UID'] = uid
event = [0] * rows
event[0] = 1
event[-1] = -1
outdf['Event'] = event

In [499]:
def expand_times2(sd,ed,unit,uid):
    outdf = {'Date' : pd.date_range(start=sd, end=pd.to_datetime(ed), freq='D', name='Date')}
    rows = len(outdf['Date'])
    outdf['Unit'] = unit
    outdf['UID'] = uid
    event = [0] * rows
    event[0] = 1
    event[-1] = -1
    outdf['Event'] = event
    return outdf

In [500]:
def expand_times3(sd,ed,unit,uid):
    outdf = {'Date' : pd.date_range(start=sd, end=pd.to_datetime(ed), freq='D', name='Date')}
    rows = len(outdf['Date'])
    outdf['Unit'] = unit
    outdf['UID'] = uid
    event = [0] * rows
    event[0] = 1
    event[-1] = -1
    outdf['Event'] = event
    return pd.DataFrame(outdf)

In [501]:
def expand_times4(sd,ed,unit,uid):
    outdf = {'Date' : pd.date_range(start=sd, end=pd.to_datetime(ed), freq='D', name='Date')}
    rows = len(outdf['Date'])
    outdf['Unit'] = unit
    outdf['UID'] = uid
    event = [0] * rows
    event[0] = 1
    event[-1] = -1
    outdf['Event'] = event
    return pd.DataFrame.from_dict(outdf)

In [502]:
def ttt():
    tdf = [expand_times2(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()]
    tdf = pd.concat([pd.DataFrame(df) for df in tdf])
    return tdf

In [503]:
%lprun -f expand_times pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()])

In [504]:
%lprun -f ttt ttt()

In [509]:
%lprun -f expand_times3 pd.concat([expand_times3(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()])

In [510]:
%lprun -f expand_times4 pd.concat([expand_times4(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df[0:1000].iterrows()])

In [511]:
# EXPAND TIMES 3 IS FASTEST

In [512]:
start = time.time()
tdf = pd.concat([expand_times3(r[start_col], r[end_col], r[unit_col], r[uid_col]) for i,r in uh_df.iterrows()])
end = time.time()
print(end - start)

128.4626109600067


In [513]:
# Month test
sd = pd.Period('2005-05-06').asfreq('M').to_timestamp()
ed = pd.Period('2005-05-25').asfreq('M').to_timestamp()

In [514]:
sd

Timestamp('2005-05-01 00:00:00')

In [515]:
pd.date_range(start=sd, end=ed, freq='D', name='Date')

DatetimeIndex(['2005-05-01'], dtype='datetime64[ns]', name='Date', freq='D')

In [516]:
pd.date_range(start=sd, end=sd, freq='MS', name='Date')

DatetimeIndex(['2005-05-01'], dtype='datetime64[ns]', name='Date', freq='MS')

In [517]:
pd.date_range(start=sd, freq='MS', name='Date', periods=1)

DatetimeIndex(['2005-05-01'], dtype='datetime64[ns]', name='Date', freq='MS')

In [534]:
def to_start_date(dates, delta_type):
    dtype = {'MS':'M', 'AS' : 'Y', 'D' : 'D'}
    delta_type = dtype[delta_type]
    plus = np.array([1], dtype='timedelta64[{}]'.format(delta_type))
    dates = dates.values.astype('datetime64[{}]'.format(delta_type)) - plus + plus
    return dates

In [535]:
mdf = uh_df[0:1000].copy()
tdf = uh_df[0:1000].copy()

In [536]:
tdf.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
1,2016-10-10,2016-06-05,1,3.0
2,2016-10-10,2007-03-29,2,15.0
6,2016-10-10,1976-08-21,3,543.0
9,2005-05-25,1996-09-12,4,11.0


In [537]:
mdf.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
1,2016-10-10,2016-06-05,1,3.0
2,2016-10-10,2007-03-29,2,15.0
6,2016-10-10,1976-08-21,3,543.0
9,2005-05-25,1996-09-12,4,11.0


In [538]:
mdf['Start.Date'] = to_start_date(mdf['Start.Date'], 'MS')
mdf['End.Date'] = to_start_date(mdf['End.Date'], 'MS')

In [539]:
mdf.head()

,End.Date,Start.Date,UID,Unit
0,2016-06-01,2007-03-01,1,14.0
1,2016-10-01,2016-06-01,1,3.0
2,2016-10-01,2007-03-01,2,15.0
6,2016-10-01,1976-08-01,3,543.0
9,2005-05-01,1996-09-01,4,11.0


In [540]:
def expand_times(sd,ed,unit,uid,freq):
    outdict = {'Date' : pd.date_range(start=sd, end=ed, freq=freq, name='Date')}
    event = [0] * len(outdict['Date'])
    event[0] = 1
    event[-1] = 2
    outdict['Event'] = event
    outdict['Unit'] = unit
    outdict['UID'] = uid
    return pd.DataFrame(outdict)

In [541]:
start = time.time()
%lprun -f expand_times pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col], 'MS') for i,r in mdf.iterrows()])
end = time.time()
print(end - start)

11.993949890136719


In [542]:
start = time.time()
full_df = pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col], 'MS') for i,r in mdf.iterrows()])
end = time.time()
print(end - start)

5.35147500038147


In [624]:
def history_to_panel(hist_df, frequency, max_date, min_date,
                     start_col='Start.Date', end_col='End.Date', unit_col='Unit', uid_col='UID'):
    
    hist_df.replace(start_col, pd.to_datetime(hist_df[start_col]), inplace=True) 
    hist_df.replace(end_col, pd.to_datetime(hist_df[end_col]), inplace=True) 
    
    if max_date:
        max_date = pd.to_datetime(max_date)
        assert isinstance(max_date, pd.Timestamp), 'max_date given is not a proper date'
    else:
        max_date = pd.to_datetime(max(hist_df[start_col]))

    hist_df = hist_df[hist_df[start_col] <= max_date]
    hist_df.loc[(hist_df[end_col].isnull()) |
              (hist_df[end_col] > max_date),
               end_col] = max_date

    if min_date:
        min_date = pd.to_datetime(min_date)
        assert isinstance(min_date, pd.Timestamp), 'min_date given is not a proper date'
        hist_df = hist_df[hist_df[end_col] >= min_date]

    hist_df = add_date_bounds(hist_df,
                              max_date=max_date, min_date=min_date,
                              start_col=start_col, end_col=end_col)
    
    freq_dict = {'Month':'MS', 'Day':'D', 'Year':'AS'}
    
    assert frequency in list(freq_dict.keys()), "freq must be 'Day', 'Month', or 'Year'"
    freq = freq_dict[frequency]
    
    hist_df[start_col] = to_start_date(hist_df[start_col], freq)
    hist_df[end_col] = to_start_date(hist_df[end_col], freq)
    
    easy_df = hist_df[hist_df[start_col] == hist_df[end_col]].copy()
    easy_df['Event'] = 3
    easy_df = easy_df[[start_col, 'Event', unit_col, uid_col]].drop_duplicates()
    easy_df.rename(columns={start_col: frequency}, inplace=True)
    
    hist_df = hist_df[hist_df[start_col] != hist_df[end_col]]
    print('Generating panel data. May take a few minutes.')
    start_time = time.time()
    panel_df = pd.concat([expand_times(r[start_col], r[end_col], r[unit_col], r[uid_col], freq)
                          for i,r in hist_df.iterrows()])
    end_time = time.time()
    print('Generating panel data took {} seconds.'.format(round(end_time-start_time, 2)))
    panel_df.rename(columns={'Date': frequency}, inplace=True)

    panel_df = panel_df.append(easy_df)
    
    if min_date:
        panel_df = panel_df[panel_df[frequency] >= min_date]
    
    return panel_df

In [625]:
history_to_panel(full_uh_df, 'Year', '2010-03-01', '2010-01-01',start_col='Start.Date', end_col='End.Date')

/Users/invinst/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Generating panel data. May take a few minutes.
Generating panel data took 33.76 seconds.


,Event,UID,Unit,Year
3,2,1,14.0,2010-01-01
3,2,2,15.0,2010-01-01
34,2,3,543.0,2010-01-01
13,2,7,12.0,2010-01-01
20,2,8,132.0,2010-01-01
21,2,10,151.0,2010-01-01
4,2,12,50.0,2010-01-01
16,2,13,50.0,2010-01-01
1,2,16,1.0,2010-01-01
1,2,17,6.0,2010-01-01


In [584]:
tdf_d = tdf.copy()
tdf_m = tdf.copy()
tdf_y = tdf.copy()

In [602]:
tm_pdf = history_to_panel(tdf_m, 'Month')

TypeError: history_to_panel() missing 2 required positional arguments: 'max_date' and 'min_date'

In [586]:
tm_pdf.head()

,Event,Month,UID,Unit
0,1,2007-03-01,1,14.0
1,0,2007-04-01,1,14.0
2,0,2007-05-01,1,14.0
3,0,2007-06-01,1,14.0
4,0,2007-07-01,1,14.0


In [587]:
td_pdf = history_to_panel(tdf_d, 'Day')

Generating panel data. May take a few minutes.
Running took 1.72 seconds.


In [588]:
td_pdf.head()

,Day,Event,UID,Unit
0,2007-03-29,1,1,14.0
1,2007-03-30,0,1,14.0
2,2007-03-31,0,1,14.0
3,2007-04-01,0,1,14.0
4,2007-04-02,0,1,14.0


In [589]:
ty_pdf = history_to_panel(tdf_y, 'Year')

Generating panel data. May take a few minutes.
Running took 1.35 seconds.


In [590]:
ty_pdf[ty_pdf['UID']==1]

,Event,UID,Unit,Year
0,1,1,14.0,2007-01-01
1,0,1,14.0,2008-01-01
2,0,1,14.0,2009-01-01
3,0,1,14.0,2010-01-01
4,0,1,14.0,2011-01-01
5,0,1,14.0,2012-01-01
6,0,1,14.0,2013-01-01
7,0,1,14.0,2014-01-01
8,0,1,14.0,2015-01-01
9,2,1,14.0,2016-01-01


In [592]:
d_pdf = history_to_panel(uh_df, 'Day')

Generating panel data. May take a few minutes.
Generating panel data took 137.48 seconds.


In [597]:
d_pdf.shape

(166137999, 4)

In [593]:
d_pdf.head()

,Day,Event,UID,Unit
0,2007-03-29,1,1,14.0
1,2007-03-30,0,1,14.0
2,2007-03-31,0,1,14.0
3,2007-04-01,0,1,14.0
4,2007-04-02,0,1,14.0


In [599]:
m_pdf = history_to_panel(uh_df, 'Month')

Generating panel data. May take a few minutes.
Generating panel data took 250.44 seconds.


In [596]:
m_pdf.shape

(5511834, 4)

In [595]:
y_pdf = history_to_panel(uh_df, 'Year')

Generating panel data. May take a few minutes.
Generating panel data took 74.46 seconds.


In [598]:
y_pdf.shape

(504473, 4)